In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load Data

In [ ]:
from pickle import load, dump

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
with open('/content/drive/MyDrive/BRIBRAIN/Codes/Post-Training/dataset/splitted_financial_news_v3.pkl', 'rb') as f:
  financial_news = load(f)

financial_news

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'url'],
        num_rows: 700
    })
    test: Dataset({
        features: ['id', 'text', 'url'],
        num_rows: 175
    })
})

# Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

In [ ]:
def preprocess_function(examples):
    # return tokenizer([" ".join(x) for x in examples["text"]])
    return tokenizer(examples["text"])

In [ ]:
tokenized_data = financial_news.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=financial_news["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/700 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/175 [00:00<?, ? examples/s]

In [ ]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_data.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/700 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/175 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# Train

In [ ]:
from transformers import AutoModelForMaskedLM, Trainer, TrainingArguments

model = AutoModelForMaskedLM.from_pretrained("indobenchmark/indobert-base-p1")

Some weights of BertForMaskedLM were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="mlm-20230416-003-2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Cloning https://huggingface.co/intanm/mlm-20230416-003-2 into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,4.415904
2,5.066900,3.768350
3,5.066900,3.444494
4,3.680300,3.236236
5,3.187200,3.040035
6,3.187200,2.939590
7,2.931900,2.863887
8,2.753000,2.767568
9,2.753000,2.701770
10,2.604100,2.669844


In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 10.18


In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/475M [00:00<?, ?B/s]

Upload file runs/Apr16_13-09-39_32d0ff88642c/events.out.tfevents.1681652979.32d0ff88642c.257.2:   0%|         …

Upload file runs/Apr16_13-09-39_32d0ff88642c/events.out.tfevents.1681650592.32d0ff88642c.257.0:   0%|         …

To https://huggingface.co/intanm/mlm-20230416-003-1
   83eadeb..4f0585d  main -> main

   83eadeb..4f0585d  main -> main

To https://huggingface.co/intanm/mlm-20230416-003-1
   4f0585d..f76b418  main -> main

   4f0585d..f76b418  main -> main



'https://huggingface.co/intanm/mlm-20230416-003-1/commit/4f0585dc2aac3631afd675d956c87a5c759dfcbf'